# Библиотека requests

In [1]:
import requests

In [ ]:
# метод get
res = requests.get('https://yandex.ru')
res
# res.status_code

In [ ]:
# браузер отрисовал бы страницу на основе данного текста
res.text

In [ ]:
# посмотрим куки
res.cookies

In [ ]:
# получаем плохой статус
bad_request = requests.get('https://yandex.ru/secret')
bad_request

In [ ]:
bad_request.text

In [ ]:
# попроуем сделать post запрос
post_req = requests.post('https://yandex.ru')
post_req

In [ ]:
# cформируем поисковый запрос, обратите внимание на его формат
URL = 'https://yandex.ru/search/?text=python&lr=54'

In [ ]:
req = requests.get(URL)
req

In [ ]:
req.text

In [ ]:
# в request можно передать параметры запроса и заголовки (headers) в виде словарей. 
# сегодня не будем рассматривать примеры с необхожимостью передачи заголовка, 
# но в практике вам это точно понадобится
URL = 'https://yandex.ru/search/'
params = {
    'text': 'python',
    'lr': 54
}
headers = {}
req = requests.get(URL, params)
req = requests.post(URL, params, headers)
req.text

In [ ]:
# очень часто сайты могут ограничивать частые запросы к себе, 
# поэтому нужно задерживать исполнение
import time
time.sleep(0.2)

# Beautiful Soup

In [5]:
# как разбирать всю эту разметку? Поможет BeautifulSoup.
from bs4 import BeautifulSoup

## Практика 1. Напишем скрипт, который будет отбирать посты из нужных хабов на habr.com

In [2]:
# определяем список хабов, которые нам интересны
DESIRED_HUBS = ['python', 'bigdata']

In [6]:
# получаем страницу с самыми свежими постами
req = requests.get('https://habr.com/ru/all/')
soup = BeautifulSoup(req.text, 'html.parser')

In [7]:
# извлекаем посты
posts = soup.find_all('article', class_='post')
posts

[<article class="post post_preview" lang="ru">
 <header class="post__meta">
 <a class="post__user-info user-info" href="https://habr.com/ru/users/Sub-ZeroMKX/" title="Автор публикации">
 <img class="user-info__image-pic user-info__image-pic_small" height="24" src="//habrastorage.org/getpro/habr/avatars/8c1/d96/e64/8c1d96e64fa99c01b15827b716a27dad.jpg" width="24"/>
 <span class="user-info__nickname user-info__nickname_small">Sub-ZeroMKX</span>
 </a>
 <span class="post__time">сегодня в 16:44</span>
 </header>
 <h2 class="post__title">
 <a class="post__title_link" href="https://habr.com/ru/post/535918/">Как стать этичным хакером в 2021 году</a>
 </h2>
 <ul class="post__hubs inline-list">
 <li class="inline-list__item inline-list__item_hub">
 <a class="inline-list__item-link hub-link" href="https://habr.com/ru/hub/infosecurity/" onclick="if (typeof ga === 'function') { ga('send', 'event', 'hub', 'feed page', 'Информационная безопасность'); }" rel="nofollow" title="Вы не подписаны на этот х

In [8]:
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    print('post', post_id)
    hubs = post.find_all('a', class_='hub-link')

post 535918
post 535916
post 535906
post 535508
post 535808
post 535908
post 535904
post 535898
post 535896
post 535846
post 535882
post 535886
post 535884
post 535816
post 535878
post 535874
post 535876
post 535872
post 535854
post 535864


In [9]:
# добавляем извлечение хабов из постов, чтобы отбирать только нужные
for post in posts:
    post_id = post.parent.attrs.get('id')
   # если идентификатор не найден, это что-то странное, пропускаем
    if not post_id:
        continue
    post_id = int(post_id.split('_')[-1])
    hubs = post.find_all('a', class_='hub-link')
    for hub in hubs:
            hub_lower = hub.text.lower()
           # ищем вхождение хотя бы одного желаемого хаба
            if any([hub_lower in desired for desired in DESIRED_HUBS]):
               # пост нам интересен - делаем с ним все что захотим:
               # можно отправит в телеграм уведомление, можно на почту и т.п.
                title_element = post.find('a', class_='post__title_link')
                print(title_element.text, title_element.attrs.get('href'))
               # так как пост уже нам подошел - дальше нет смысла проверять хабы
                break

Хакатоны Осень 2020. Мой опыт https://habr.com/ru/post/535908/


## Практика 2. Напишем скрипт, который будет собирать новости с сайта Коммерсанта

In [ ]:
URL = 'https://www.kommersant.ru/search/results'
params = {
    'search_query': 'python'
}

In [ ]:
res = requests.get(URL, params)

In [ ]:
res.text

In [ ]:
soup = BeautifulSoup(res.text, 'html.parser')
soup

In [ ]:
# добираемся до блоков с новостями
news_blocks = soup.find_all('div', class_='search_results_item')
news_blocks

In [ ]:
# добираемся до текста со ссылкой
articles_intro = list(map(lambda x: x.find('div', class_='article_intro'), news_blocks))
articles_intro

In [ ]:
# добираемся до ссылок
a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
a_list

In [ ]:
# формируем полноценные ссылки
all_refs = list(map(lambda x: 'https://www.kommersant.ru/' + x, a_list))
all_refs

In [ ]:
# объединим все в одну функцию
def get_all_links(url, query):
    all_refs = []
    params = {
        'search_query': query,
    }
    res = requests.get(URL, params)
    time.sleep(0.3)
    soup = BeautifulSoup(res.text, 'html.parser')
    news_blocks = soup.find_all('div', class_='search_results_item')
    articles_intro = list(map(lambda x: x.find('div', class_='article_intro'), news_blocks))
    a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
    all_refs = list(map(lambda x: 'https://www.kommersant.ru/' + x, a_list))

    return all_refs

all_links = get_all_links(URL, 'python')
all_links

In [ ]:
# но мы же собрали только одну страницу? Хотим ВСЕ новости
def get_all_links(url, query, pages):
    all_refs = []
    params = {
        'search_query': query
    }
    for i in range(pages):
        params['page'] = i + 1
        res = requests.get(URL, params)
        time.sleep(0.3)
        soup = BeautifulSoup(res.text, 'html.parser')
        news_blocks = soup.find_all('div', class_='search_results_item')
        articles_intro = list(map(lambda x: x.find('div', class_='article_intro'), news_blocks))
        a_list = list(map(lambda x: x.find('a').get('href'), articles_intro))
        all_refs += list(map(lambda x: 'https://www.kommersant.ru/' + x, a_list))
    return all_refs

all_links = get_all_links(URL, 'python', 3)
all_links

In [17]:
import pandas as pd

In [ ]:
# собираем даты, заголовки и тексты новостей
# получаем ошибочку. Значит не у всех получаемых страниц одинаковая разметка
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    print(date)
    title = soup.find('h1', class_='article_name').text
    print(title)
    text = soup.find('div', class_='article_text_wrapper').text
    print(text)

In [ ]:
# получаем ошибочку. Значит не у всех получаемых страниц одинаковая разметка
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    if soup.find('div', class_='b-article__publish_date'):
        date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
    elif soup.find('time', class_='title__cake'):
        date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    print(date)
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='article_name').text
    else: 
        title = soup.find('h1', class_='article_name').text    
    print(title)
    text = soup.find('div', class_='article_text_wrapper').text
    print(text)

In [ ]:
# запишем данные в датафрейм
kom_news = pd.DataFrame()
for link in all_links:
    soup = BeautifulSoup(requests.get(link).text, 'html.parser')
    time.sleep(0.3)
    if soup.find('div', class_='b-article__publish_date'):
        date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
    elif soup.find('time', class_='title__cake'):
        date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
    if soup.find('h2', class_='article_name'): 
        title = soup.find('h2', class_='article_name').text
    else: 
        title = soup.find('h1', class_='article_name').text    
    text = soup.find('div', class_='article_text_wrapper').text
    row = {'date': date, 'title': title, 'text': text}
    kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
kom_news

In [ ]:
# обернем в функцию 
def get_kom_news(links):
    kom_news = pd.DataFrame()
    for link in all_links:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        if soup.find('div', class_='b-article__publish_date'):
            date = pd.to_datetime(soup.find('div', class_='b-article__publish_date').find('time').get('datetime'), dayfirst=True).date()
        elif soup.find('time', class_='title__cake'):
            date = pd.to_datetime(soup.find('time', class_='title__cake').get('datetime'), dayfirst=True).date()
        if soup.find('h2', class_='article_name'): 
            title = soup.find('h2', class_='article_name').text
        else: 
            title = soup.find('h1', class_='article_name').text    
        text = soup.find('div', class_='article_text_wrapper').text
        row = {'date': date, 'title': title, 'text': text}
        kom_news = pd.concat([kom_news, pd.DataFrame([row])])  
    return kom_news

In [ ]:
kom_news = get_kom_news(all_links)
kom_news

# API

## Практика 3. Получим данные о песнях исполнителя при помощи  [API ITunes](https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/index.html)

In [10]:
# https://developer.apple.com/library/archive/documentation/AudioVideo/Conceptual/iTuneSearchAPI/UnderstandingSearchResults.html#//apple_ref/doc/uid/TP40017632-CH8-SW1
URL = 'https://itunes.apple.com/search?term=jack+johnson'

In [11]:
res = requests.get(URL)

In [12]:
res.text

'\n\n\n{\n "resultCount":50,\n "results": [\n{"wrapperType":"track", "kind":"song", "artistId":909253, "collectionId":1469577723, "trackId":1469577741, "artistName":"Jack Johnson", "collectionName":"Jack Johnson and Friends: Sing-A-Longs and Lullabies for the Film Curious George", "trackName":"Upside Down", "collectionCensoredName":"Jack Johnson and Friends: Sing-A-Longs and Lullabies for the Film Curious George", "trackCensoredName":"Upside Down", "artistViewUrl":"https://music.apple.com/us/artist/jack-johnson/909253?uo=4", "collectionViewUrl":"https://music.apple.com/us/album/upside-down/1469577723?i=1469577741&uo=4", "trackViewUrl":"https://music.apple.com/us/album/upside-down/1469577723?i=1469577741&uo=4", \n"previewUrl":"https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview123/v4/75/5a/fc/755afca1-d911-be2b-c0fb-f99d6d52ce1a/mzaf_4156076990936187406.plus.aac.p.m4a", "artworkUrl30":"https://is4-ssl.mzstatic.com/image/thumb/Music123/v4/be/38/d0/be38d058-31ed-c0ea-91e6-120528

In [13]:
res.json()

{'resultCount': 50,
 'results': [{'wrapperType': 'track',
   'kind': 'song',
   'artistId': 909253,
   'collectionId': 1469577723,
   'trackId': 1469577741,
   'artistName': 'Jack Johnson',
   'collectionName': 'Jack Johnson and Friends: Sing-A-Longs and Lullabies for the Film Curious George',
   'trackName': 'Upside Down',
   'collectionCensoredName': 'Jack Johnson and Friends: Sing-A-Longs and Lullabies for the Film Curious George',
   'trackCensoredName': 'Upside Down',
   'artistViewUrl': 'https://music.apple.com/us/artist/jack-johnson/909253?uo=4',
   'collectionViewUrl': 'https://music.apple.com/us/album/upside-down/1469577723?i=1469577741&uo=4',
   'trackViewUrl': 'https://music.apple.com/us/album/upside-down/1469577723?i=1469577741&uo=4',
   'previewUrl': 'https://audio-ssl.itunes.apple.com/itunes-assets/AudioPreview123/v4/75/5a/fc/755afca1-d911-be2b-c0fb-f99d6d52ce1a/mzaf_4156076990936187406.plus.aac.p.m4a',
   'artworkUrl30': 'https://is4-ssl.mzstatic.com/image/thumb/Music123

In [14]:
params = {
    'term': 'шнуров',
    'limit': 200,
#     'offset': 2
}

In [18]:
# pd.set_option('display.max_columns', 100)
res = requests.get(URL, params)
# res.json()

pd.DataFrame(res.json()['results'])

,wrapperType,kind,artistId,collectionId,trackId,artistName,collectionName,trackName,collectionCensoredName,trackCensoredName,...,isStreamable,collectionArtistId,collectionArtistName,trackRentalPrice,collectionHdPrice,trackHdPrice,trackHdRentalPrice,contentAdvisoryRating,shortDescription,longDescription
0,track,song,255133049.0,1.089863e+09,1089863288,Сергей Лазарев,You Are the Only One - Single,You Are the Only One,You Are the Only One - Single,You Are the Only One,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,track,song,255133049.0,1.246353e+09,1246353561,Сергей Лазарев & Дима Билан,Прости меня - Single,Прости меня,Прости меня - Single,Прости меня,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,track,song,255133049.0,1.266230e+09,1266229819,Сергей Лазарев,Так красиво - Single,Так красиво,Так красиво - Single,Так красиво,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,track,song,255133049.0,1.017290e+09,1017290110,Сергей Лазарев,Это всё она - Single,Это всё она,Это всё она - Single,Это всё она,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,track,song,255133049.0,1.470935e+09,1470935172,Сергей Лазарев,Лови - Single,Лови,Лови - Single,Лови,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,track,song,255133049.0,1.276318e+09,1276317737,Сергей Лазарев,Сдавайся - Single,Сдавайся,Сдавайся - Single,Сдавайся,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,track,song,255133049.0,1.292270e+09,1292269887,Сергей Лазарев,Шёпотом - Single,Шёпотом,Шёпотом - Single,Шёпотом,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,track,song,255133049.0,1.477109e+09,1477109432,Сергей Лазарев,Я не боюсь,Я не боюсь,Я не боюсь,Я не боюсь,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,track,song,255133049.0,1.327033e+09,1327032972,Сергей Лазарев,Новый год - Single,Новый год,Новый год - Single,Новый год (HarDrum Remix),...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,track,song,255133049.0,9.202454e+08,920245455,Сергей Лазарев,7 Цифр - Single,7 Цифр,7 Цифр - Single,7 Цифр,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1]:
params = {
    'term': 'лазарев',
    'limit': 60,
    'attribute': 'allArtistTerm',
    'country': 'ru'
}

In [20]:
res = requests.get(URL, params)
res.json()

pd.DataFrame(res.json()['results'])

,wrapperType,kind,artistId,collectionId,trackId,artistName,collectionName,trackName,collectionCensoredName,trackCensoredName,...,isStreamable,collectionArtistId,collectionArtistName,trackRentalPrice,collectionHdPrice,trackHdPrice,trackHdRentalPrice,contentAdvisoryRating,shortDescription,longDescription
0,track,song,255133049.0,1.089863e+09,1089863288,Сергей Лазарев,You Are the Only One - Single,You Are the Only One,You Are the Only One - Single,You Are the Only One,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,track,song,255133049.0,1.246353e+09,1246353561,Сергей Лазарев & Дима Билан,Прости меня - Single,Прости меня,Прости меня - Single,Прости меня,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,track,song,255133049.0,1.266230e+09,1266229819,Сергей Лазарев,Так красиво - Single,Так красиво,Так красиво - Single,Так красиво,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,track,song,255133049.0,1.017290e+09,1017290110,Сергей Лазарев,Это всё она - Single,Это всё она,Это всё она - Single,Это всё она,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,track,song,255133049.0,1.470935e+09,1470935172,Сергей Лазарев,Лови - Single,Лови,Лови - Single,Лови,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,track,song,255133049.0,1.276318e+09,1276317737,Сергей Лазарев,Сдавайся - Single,Сдавайся,Сдавайся - Single,Сдавайся,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,track,song,255133049.0,1.292270e+09,1292269887,Сергей Лазарев,Шёпотом - Single,Шёпотом,Шёпотом - Single,Шёпотом,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,track,song,255133049.0,1.477109e+09,1477109432,Сергей Лазарев,Я не боюсь,Я не боюсь,Я не боюсь,Я не боюсь,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,track,song,255133049.0,1.327033e+09,1327032972,Сергей Лазарев,Новый год - Single,Новый год,Новый год - Single,Новый год (HarDrum Remix),...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,track,song,255133049.0,9.202454e+08,920245455,Сергей Лазарев,7 Цифр - Single,7 Цифр,7 Цифр - Single,7 Цифр,...,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


## Практика 4. Соберем сообщения из новостной ленты ВК по нужному запросу

In [ ]:
# https://vk.com/dev/manuals
# https://vk.com/dev/newsfeed.search
NEWSFEED_REQUEST = 'https://api.vk.com/method/newsfeed.search?'
TOKEN = '9df7991c9df7991c9df7991c329d86910d99df79df7991cc363a27748dcf7ad91284ef6'
VERSION = '5.103'
SLEEP = 0.33

In [ ]:
# обращаем внимание, что максимальное количество постов, 
# которые можно вытащить за раз, ограничено
params = {
    'access_token': TOKEN,
    'v': VERSION,
    'q': 'короновирус',
    'count': 200
}

In [ ]:
res = requests.get(NEWSFEED_REQUEST, params)
res

In [ ]:
res.text

In [ ]:
res.json()

In [ ]:
res.json()['response']['items']

In [ ]:
pd.DataFrame(res.json()['response']['items'])

In [ ]:
# соберем все доступные сообщения по запросу
newsfeed_df = pd.DataFrame()
while True:
    result = requests.get(NEWSFEED_REQUEST, params)
    time.sleep(0.33)
    newsfeed_df = pd.concat([newsfeed_df, pd.DataFrame(result.json()['response']['items'])])
    if 'next_from' in result.json()['response'].keys():
        params['start_from'] = result.json()['response']['next_from']
    else:
        break
newsfeed_df